In [20]:
import pandas as pd
import numpy as np

In [21]:
df = pd.read_csv("train.csv", low_memory=False)
store = pd.read_csv("store.csv")

### Data Prep

In [22]:
print('sample store data')
store.head()

sample store data


,Store,StoreType,Assortment,CompetitionDistance,CompetitionOpenSinceMonth,CompetitionOpenSinceYear,Promo2,Promo2SinceWeek,Promo2SinceYear,PromoInterval
0,1,c,a,1270.0,9.0,2008.0,0,NaN,NaN,NaN
1,2,a,a,570.0,11.0,2007.0,1,13.0,2010.0,"Jan,Apr,Jul,Oct"
2,3,a,a,14130.0,12.0,2006.0,1,14.0,2011.0,"Jan,Apr,Jul,Oct"
3,4,c,c,620.0,9.0,2009.0,0,NaN,NaN,NaN
4,5,a,a,29910.0,4.0,2015.0,0,NaN,NaN,NaN


In [23]:
print('sample sales data')
df.head()

sample sales data


,Store,DayOfWeek,Date,Sales,Customers,Open,Promo,StateHoliday,SchoolHoliday
0,1,5,2015-07-31,5263,555,1,1,0,1
1,2,5,2015-07-31,6064,625,1,1,0,1
2,3,5,2015-07-31,8314,821,1,1,0,1
3,4,5,2015-07-31,13995,1498,1,1,0,1
4,5,5,2015-07-31,4822,559,1,1,0,1


In [24]:
print ("training data", df.shape)
print ("Stores data", store.shape)

training data (1017209, 9)
Stores data (1115, 10)


### Data Type Conversion

In [27]:
# df['CompetitionOpenSinceYear'] = df['CompetitionOpenSinceYear'].np.dtype(int)
# df['CompetitionOpenSinceMonth'] = df['CompetitionOpenSinceMonth'].np.dtype(int)
# df['StateHoliday'] = df['StateHoliday'].np.dtype(str)
# df['Promo2SinceWeek'] = df['Promo2SinceWeek'].np.dtype(int)
df['SchoolHoliday'] = df['SchoolHoliday'].np.dtype(float)
df['PromoInterval'] = df['PromoInterval'].np.dtype(str)

AttributeError: 'Series' object has no attribute 'np'

In [ ]:
Q1 = np.percentile(df['Sales'], 25)
Q3 = np.percentile(df['Sales'], 75)
step = step = 1.5 * (Q3 - Q1)
i=df[~((df['Sales'] >= Q1 - step) & (df['Sales'] <= Q3 + step))].index
Q1 = np.percentile(df['Customers'], 25)
Q3 = np.percentile(df['Customers'], 75)
step = step = 1.5 * (Q3 - Q1)
j=df[~((df['Customers'] >= Q1 - step) & (df['Customers'] <= Q3 + step))].index
common_outliers = set(i).intersection(j)

In [19]:
df = df.drop(df.index[list(common_outliers)]).reset_index(drop = True)
print (float(len(common_outliers))*100/float(df.shape[0]), "%  of data dropped")

0.02269135539682763 %  of data dropped


In [ ]:
# remove NaNs-
data.fillna(0, inplace=True)
#take the store as open if no value given-
data.loc[data.Open.isnull(), 'Open'] = 1
#only process stores which are open, otherwise whats the point:
data = data[data["Open"] != 0]         
# Use some properties directly
features.extend(['Store', 'CompetitionDistance', 'Promo', 'Promo2', 'SchoolHoliday'])
# Label encode some features
features.extend(['StoreType', 'Assortment', 'StateHoliday'])
mappings = {'0':0, 'a':1, 'b':2, 'c':3, 'd':4}
data.StoreType.replace(mappings, inplace=True)
data.Assortment.replace(mappings, inplace=True)
data.StateHoliday.replace(mappings, inplace=True)

features.extend(['DayOfWeek', 'Month', 'Day', 'Year', 'WeekOfYear'])
data['Year'] = data.Date.dt.year
data['Month'] = data.Date.dt.month
data['Day'] = data.Date.dt.day
data['DayOfWeek'] = data.Date.dt.dayofweek
data['WeekOfYear'] = data.Date.dt.weekofyear


# Calculate time competition open time in months
features.append('CompetitionOpen')
data['CompetitionOpen'] = 12 * (data.Year - data.CompetitionOpenSinceYear) + (data.Month - data.CompetitionOpenSinceMonth)
# Promo open time in months
features.append('PromoOpen')
data['PromoOpen'] = 12 * (data.Year - data.Promo2SinceYear) + (data.WeekOfYear - data.Promo2SinceWeek) / 4.0
data['PromoOpen'] = data.PromoOpen.apply(lambda x: x if x > 0 else 0)
data.loc[data.Promo2SinceYear == 0, 'PromoOpen'] = 0

# Indicate that sales on that day are in promo interval
features.append('IsPromoMonth')
month2str = {1:'Jan', 2:'Feb', 3:'Mar', 4:'Apr', 5:'May', 6:'Jun', 7:'Jul', 8:'Aug', 9:'Sept', 10:'Oct', 11:'Nov', 12:'Dec'}
data['monthStr'] = data.Month.map(month2str)
data.loc[data.PromoInterval == 0, 'PromoInterval'] = ''
data['IsPromoMonth'] = 0
for interval in data.PromoInterval.unique():
    if interval != '':
        for month in interval.split(','):
            data.loc[(data.monthStr == month) & (data.PromoInterval == interval), 'IsPromoMonth'] = 1
